In [1]:
!pip install kafka-python requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.3/326.3 kB 2.4 MB/s eta 0:00:00a 0:00:01


In [1]:
import requests
import json
import time
from kafka import KafkaProducer

API_URL = "https://api.open-meteo.com/v1/forecast"
LAT, LON = 43.610769, 3.876716 # MPL

KAFKA_TOPIC = "weather_transformed"
KAFKA_BROKER = "kafka:9092"

def fetch_weather():
    params = { "latitude": LAT, "longitude": LON, "current_weather": "true" }
    resp = requests.get(API_URL, params=params, timeout=10)
    resp.raise_for_status()
    return resp.json().get("current_weather", {})

def transform_weather(record: dict) -> dict:
    # Convertir Celsius en Fahrenheit
    if "temperature" in record:
        record["temp_f"] = record["temperature"] * 9/5 + 32
    # Alerte vent fort
    record["high_wind_alert"] = record.get("windspeed", 0) > 10 
    return record

def main():
    producer = KafkaProducer(
        bootstrap_servers=KAFKA_BROKER, 
        value_serializer=lambda v: json.dumps(v).encode("utf-8"))
    print("Weather streaming producer started...")
    while True: 
        try:
            weather = fetch_weather() 
            if weather:
                transformed = transform_weather(weather)
                producer.send(KAFKA_TOPIC, transformed)
                producer.flush()
                print("Sent to weather_transformed:", transformed)
        except Exception as e:
            print("Error fetching or sending weather:", e)
        time.sleep(30) # fetch every 30 seconds
        

if __name__ == "__main__": 
    main()

Weather streaming producer started...
Sent to weather_transformed: {'time': '2026-01-22T08:45', 'interval': 900, 'temperature': 7.1, 'windspeed': 21.4, 'winddirection': 7, 'is_day': 1, 'weathercode': 63, 'temp_f': 44.78, 'high_wind_alert': True}
Sent to weather_transformed: {'time': '2026-01-22T08:45', 'interval': 900, 'temperature': 7.1, 'windspeed': 21.4, 'winddirection': 7, 'is_day': 1, 'weathercode': 63, 'temp_f': 44.78, 'high_wind_alert': True}
Sent to weather_transformed: {'time': '2026-01-22T08:45', 'interval': 900, 'temperature': 7.1, 'windspeed': 21.4, 'winddirection': 7, 'is_day': 1, 'weathercode': 63, 'temp_f': 44.78, 'high_wind_alert': True}
Sent to weather_transformed: {'time': '2026-01-22T08:45', 'interval': 900, 'temperature': 7.1, 'windspeed': 21.4, 'winddirection': 7, 'is_day': 1, 'weathercode': 63, 'temp_f': 44.78, 'high_wind_alert': True}
Sent to weather_transformed: {'time': '2026-01-22T08:45', 'interval': 900, 'temperature': 7.1, 'windspeed': 21.4, 'winddirection':

KeyboardInterrupt: 